In [567]:
import numpy as np
import pandas as pd
import scipy
import re
import matplotlib.pyplot as plt
import string
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from nltk.corpus import stopwords

#Step 1 Reading the training dataset
train = pd.read_csv("train.csv")


train.head()

,sentiment,text
0,5,@manjulamartin @Kirk_Gleason Except trains are...
1,5,I want a Google driverless car.
2,5,@Oatmeal @google driverless @TeslaMotors ? Ooo...
3,5,SO MUCH AWESOME! Amazing video for GoogleÌ¢‰âÂ...
4,5,@google is making driverless cars which is awe...


In [568]:
import re
#Step 2= Removing Twitter Handles

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 

# creating new column tidy_tweet & remove twitter handles (@user)
train['clean_text'] = np.vectorize(remove_pattern)(train['text'], "@[\w]*")

train['clean_text'] = train['clean_text'].str.replace('http\S+|www.\S+', '', case=False)

#Step 3= Removing Punctuations, Numbers, and Special Character 
train['clean_text'] = train['clean_text'].str.replace('[^A-Za-z0-9]+', ' ')
                          
# Step 4 remove stopwords
train['clean_text'] = train['clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

#Step 5 to lowercase

train['clean_text']= train['clean_text'].apply(lambda x: ''.join([w for w in x.lower()]))

train['clean_text'].head(20)

0     except trains very capital intensive have limi...
1                                want google driverless
2                   driverless ooooh that would awesome
3     much awesome amazing video google next phase d...
4                  making driverless cars which awesome
5       could call driverless tell will take there whoo
6                           google driverless cars take
7     driverless taxis would probably amazing applic...
8         this whole driverless prototype thing awesome
9     google driverless brake pedal steering wheel t...
10    riding driverless bucket list this damn exciti...
11      this future places will wait driverless scichat
12    driverless great idea someone like condition t...
13                            this need driverless cars
14       google developed driverless that fucking chill
15    wait driverless cars soon rarely drive many li...
16                     they coming with driverless cars
17                                      want dri

In [569]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train['clean_text'], train['sentiment'],test_size = 0.2)


## for transforming the 80% of the train data ##

from sklearn.feature_extraction.text import CountVectorizer as CountVectorizer
count_vect = CountVectorizer(stop_words='english')
x_train_counts = count_vect.fit_transform(x_train)


from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
x_train_tfidf = transformer.fit_transform(x_train_counts)
x_train_tfidf.shape

## for transforming the 20% of the train data which is being used for validation ##
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)



from sklearn.ensemble.forest import RandomForestClassifier
model = RandomForestClassifier(n_estimators=980) # >>>>>typo error <<<< :  'i' went missing in RandomForestClassifier
model.fit(x_train_tfidf,y_train)

prediction = model.predict(x_test_tfidf)
print(prediction) # >>>typo error <<<  : 's'was present at the end of prediction

[3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 4 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 4 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 4 3 3 3 3 3 3 3]


In [570]:
# testing model's accuracy with accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,prediction))

0.6802030456852792


In [571]:
from sklearn.metrics import confusion_matrix
print('confusion matrix: \n',confusion_matrix(y_test, prediction))

confusion matrix: 
 [[  0   0   4   0   0]
 [  0   0  14   0   0]
 [  1   0 128   1   0]
 [  1   0  31   5   0]
 [  0   0  10   1   1]]


In [572]:
from sklearn.metrics import classification_report
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00        14
           3       0.68      0.98      0.81       130
           4       0.71      0.14      0.23        37
           5       1.00      0.08      0.15        12

   micro avg       0.68      0.68      0.68       197
   macro avg       0.48      0.24      0.24       197
weighted avg       0.65      0.68      0.58       197



/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [573]:
from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(train['clean_text'], train['sentiment'], test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)


In [574]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils


In [575]:
def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

In [576]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [577]:
from gensim.models.phrases import Phrases
from gensim.models.phrases import Phraser

In [578]:
tokenized_train = [t.split() for t in x_train]

In [579]:
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)

In [580]:
def labelize_tweets_bg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(bigram[t.split()], [prefix + '_%s' % i]))
    return result

In [581]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v_bg = labelize_tweets_bg(all_x, 'all')

In [582]:
cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_x_w2v_bg)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 981/981 [00:00<00:00, 1085936.19it/s]


In [583]:
#dbow

train_vecs_dbow_bg = get_vectors(model_bg_dbow, x_train, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, x_validation, 100)

In [584]:
clf = RandomForestClassifier(n_estimators=980)
clf.fit(train_vecs_dbow_bg, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=980, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [585]:
clf.score(validation_vecs_dbow_bg, y_validation)

0.6

In [586]:
from sklearn.linear_model import LogisticRegression

In [587]:
clf2 = model2 = LogisticRegression()
clf2.fit(train_vecs_dbow_bg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [588]:
clf2.score(validation_vecs_dbow_bg, y_validation)

0.6

In [589]:
from sklearn import tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_bg, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [590]:
clf3.score(validation_vecs_dbow_bg, y_validation)

0.2

In [591]:
#SVM
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_bg, y_train)
clf4.score(validation_vecs_dbow_bg, y_validation)

0.1

In [592]:
# K-NN Classifier
from sklearn.neighbors import KNeighborsClassifier
clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_bg, y_train)
clf5.score(validation_vecs_dbow_bg, y_validation)

0.6

In [593]:
model_bg_dbow.save('d2v_model_bg_dbow.doc2vec')
model_bg_dbow = Doc2Vec.load('d2v_model_bg_dbow.doc2vec')
model_bg_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [594]:
#DMC Bigram

In [595]:
cores = multiprocessing.cpu_count()
model_bg_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmc.build_vocab([x for x in tqdm(all_x_w2v_bg)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 981/981 [00:00<00:00, 644519.46it/s]


In [596]:
for epoch in range(30):
    model_bg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmc.alpha -= 0.002
    model_bg_dmc.min_alpha = model_bg_dmc.alpha

100%|██████████| 981/981 [00:00<00:00, 721861.79it/s]


In [597]:
model_bg_dmc.most_similar('driverless')

[('cars', 0.8473590016365051),
 ('action', 0.7662856578826904),
 ('developing', 0.6675175428390503),
 ('robot', 0.6353542804718018),
 ('driver_less', 0.6329611539840698),
 ('powered', 0.6269574165344238),
 ('flying', 0.6230174899101257),
 ('them', 0.622847855091095),
 ('based', 0.6149864792823792),
 ('mass', 0.6097685098648071)]

In [598]:
model_bg_dmc.most_similar(positive=['car', 'driverless'], negative=['driver'], topn=5)


[('action', 0.6619821786880493),
 ('electric', 0.5447269678115845),
 ('mass', 0.5143768787384033),
 ('transit', 0.49883246421813965),
 ('allow', 0.4960136115550995)]

In [599]:
train_vecs_dmc_bg = get_vectors(model_bg_dmc, x_train, 100)
validation_vecs_dmc_bg = get_vectors(model_bg_dmc, x_validation, 100)

In [600]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc_bg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [601]:
clf.score(validation_vecs_dmc_bg, y_validation)

0.6

In [602]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dmc_bg, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=980, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [603]:
clf2.score(validation_vecs_dmc_bg, y_validation)

0.7

In [604]:
#tree
from sklearn import tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dmc_bg, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [605]:
clf3.score(validation_vecs_dmc_bg, y_validation)

0.7

In [606]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dmc_bg, y_train)
clf4.score(validation_vecs_dmc_bg, y_validation)

0.5

In [607]:
clf4.score(validation_vecs_dmc_bg, y_validation)

0.5

In [608]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dmc_bg, y_train)
clf5.score(validation_vecs_dmc_bg, y_validation)

0.7

In [609]:
#DMM BIgram

In [610]:
cores = multiprocessing.cpu_count()
model_bg_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dmm.build_vocab([x for x in tqdm(all_x_w2v_bg)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 981/981 [00:00<00:00, 558367.79it/s]


In [611]:
for epoch in range(30):
    model_bg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_bg)]), total_examples=len(all_x_w2v_bg), epochs=1)
    model_bg_dmm.alpha -= 0.002
    model_bg_dmm.min_alpha = model_bg_dmm.alpha

100%|██████████| 981/981 [00:00<00:00, 1502780.21it/s]


In [612]:
train_vecs_dmm_bg = get_vectors(model_bg_dmm, x_train, 100)
validation_vecs_dmm_bg = get_vectors(model_bg_dmm, x_validation, 100)

In [613]:
clf = LogisticRegression()
clf.fit(train_vecs_dmm_bg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [614]:
clf.score(validation_vecs_dmm_bg, y_validation)

0.6

In [615]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dmm_bg, y_train)
clf2.score(validation_vecs_dmm_bg, y_validation)

0.7

In [616]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dmm_bg, y_train)
clf3.score(validation_vecs_dmm_bg, y_validation)

0.5

In [617]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dmm_bg, y_train)
clf4.score(validation_vecs_dmm_bg, y_validation)

0.5

In [618]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dmm_bg, y_train)
clf5.score(validation_vecs_dmm_bg, y_validation)

0.6

In [619]:
model_bg_dmc.save('d2v_model_bg_dmc.doc2vec')
model_bg_dmc = Doc2Vec.load('d2v_model_bg_dmc.doc2vec')
model_bg_dmc.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [620]:
train_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_train, 200)
validation_vecs_dbow_dmc_bg = get_concat_vectors(model_bg_dbow,model_bg_dmc, x_validation, 200)

In [621]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_bg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [622]:
clf.score(validation_vecs_dbow_dmc_bg, y_validation)

0.6

In [623]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dbow_dmc_bg, y_train)
clf2.score(validation_vecs_dbow_dmc_bg, y_validation)

0.6

In [624]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_dmc_bg, y_train)
clf3.score(validation_vecs_dbow_dmc_bg, y_validation)

0.5

In [625]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_dmc_bg, y_train)
clf4.score(validation_vecs_dbow_dmc_bg, y_validation)

0.5

In [626]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_dmc_bg, y_train)
clf5.score(validation_vecs_dbow_dmc_bg, y_validation)

0.7

In [627]:
train_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_train, 200)
validation_vecs_dbow_dmm_bg = get_concat_vectors(model_bg_dbow,model_bg_dmm, x_validation, 200)

In [628]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_bg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [629]:
clf.score(validation_vecs_dbow_dmm_bg, y_validation)

0.6

In [630]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dbow_dmm_bg, y_train)
clf2.score(validation_vecs_dbow_dmm_bg, y_validation)

0.6

In [631]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_dmm_bg, y_train)
clf3.score(validation_vecs_dbow_dmm_bg, y_validation)

0.6

In [632]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_dmm_bg, y_train)
clf4.score(validation_vecs_dbow_dmm_bg, y_validation)

0.5

In [633]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_dmm_bg, y_train)
clf5.score(validation_vecs_dbow_dmm_bg, y_validation)

0.6

In [634]:
#trigram

In [635]:
tg_phrases = Phrases(bigram[tokenized_train])
trigram = Phraser(tg_phrases)

In [636]:
def labelize_tweets_tg(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(trigram[bigram[t.split()]], [prefix + '_%s' % i]))
    return result

In [637]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v_tg = labelize_tweets_tg(all_x, 'all')

In [638]:
model_tg_dbow = Doc2Vec(dm=0, size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dbow.build_vocab([x for x in tqdm(all_x_w2v_tg)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 981/981 [00:00<00:00, 424624.58it/s]


In [639]:
for epoch in range(30):
    model_tg_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dbow.alpha -= 0.002
    model_tg_dbow.min_alpha = model_tg_dbow.alpha

100%|██████████| 981/981 [00:00<00:00, 1858451.77it/s]


In [640]:
train_vecs_dbow_tg = get_vectors(model_tg_dbow, x_train, 100)
validation_vecs_dbow_tg = get_vectors(model_tg_dbow, x_validation, 100)

In [641]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_tg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [642]:
clf.score(validation_vecs_dbow_tg, y_validation)

0.6

In [643]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dbow_tg, y_train)
clf2.score(validation_vecs_dbow_tg, y_validation)

0.6

In [644]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_tg, y_train)
clf3.score(validation_vecs_dbow_tg, y_validation)

0.3

In [645]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_tg, y_train)
clf4.score(validation_vecs_dbow_tg, y_validation)

0.5

In [646]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_tg, y_train)
clf5.score(validation_vecs_dbow_tg, y_validation)

0.6

In [647]:
model_tg_dbow.save('d2v_model_tg_dbow.doc2vec')
model_tg_dbow = Doc2Vec.load('d2v_model_tg_dbow.doc2vec')
model_tg_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [648]:
#DMC Trigram

In [649]:
cores = multiprocessing.cpu_count()
model_tg_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmc.build_vocab([x for x in tqdm(all_x_w2v_tg)])

/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 981/981 [00:00<00:00, 1058285.04it/s]


In [650]:
for epoch in range(30):
    model_tg_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmc.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

100%|██████████| 981/981 [00:00<00:00, 1754632.08it/s]


In [651]:
train_vecs_dmc_tg = get_vectors(model_tg_dmc, x_train, 100)
validation_vecs_dmc_tg = get_vectors(model_tg_dmc, x_validation, 100)

In [652]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc_tg, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [653]:
clf.score(validation_vecs_dmc_tg, y_validation)

0.6

In [654]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dmc_tg, y_train)
clf2.score(validation_vecs_dmc_tg, y_validation)

0.6

In [ ]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dmc_tg, y_train)
clf3.score(validation_vecs_dmc_tg, y_validation)

0.6

In [ ]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dmc_tg, y_train)
clf4.score(validation_vecs_dmc_tg, y_validation)

In [ ]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dmc_tg, y_train)
clf5.score(validation_vecs_dmc_tg, y_validation)

In [ ]:
model_tg_dmc.save('d2v_model_tg_dmc.doc2vec')
model_tg_dmc = Doc2Vec.load('d2v_model_tg_dmc.doc2vec')
model_tg_dmc.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [ ]:
#DMM trigram

In [ ]:
cores = multiprocessing.cpu_count()
model_tg_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_tg_dmm.build_vocab([x for x in tqdm(all_x_w2v_tg)])

In [ ]:
for epoch in range(30):
    model_tg_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v_tg)]), total_examples=len(all_x_w2v_tg), epochs=1)
    model_tg_dmm.alpha -= 0.002
    model_tg_dmc.min_alpha = model_tg_dmc.alpha

In [ ]:
train_vecs_dmm_tg = get_vectors(model_tg_dmm, x_train, 100)
validation_vecs_dmm_tg = get_vectors(model_tg_dmm, x_validation, 100)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dmm_tg, y_train)

In [ ]:
clf.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dmm_tg, y_train)
clf2.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dmm_tg, y_train)
clf3.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dmm_tg, y_train)
clf4.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dmm_tg, y_train)
clf5.score(validation_vecs_dmm_tg, y_validation)

In [ ]:
model_tg_dmm.save('d2v_model_tg_dmm.doc2vec')
model_tg_dmm = Doc2Vec.load('d2v_model_tg_dmm.doc2vec')
model_tg_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [ ]:
train_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_train, 200)
validation_vecs_dbow_dmc_tg = get_concat_vectors(model_tg_dbow,model_tg_dmc, x_validation, 200)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc_tg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dbow_dmc_tg, y_train)
clf2.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_dmc_tg, y_train)
clf3.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_dmc_tg, y_train)
clf4.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_dmc_tg, y_train)
clf5.score(validation_vecs_dbow_dmc_tg, y_validation)

In [ ]:
train_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_train, 200)
validation_vecs_dbow_dmm_tg = get_concat_vectors(model_tg_dbow,model_tg_dmm, x_validation, 200)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm_tg, y_train)

In [ ]:
clf.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_dbow_dmc_tg, y_train)
clf2.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_dbow_dmc_tg, y_train)
clf3.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_dbow_dmm_tg, y_train)
clf4.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_dbow_dmm_tg, y_train)
clf5.score(validation_vecs_dbow_dmm_tg, y_validation)

In [ ]:
model_tg_dmm.save('d2v_model_ug_dbow.doc2vec')
model_ug_dbow = Doc2Vec.load('d2v_model_ug_dbow.doc2vec')
model_tg_dmm = Doc2Vec.load('d2v_model_tg_dmm.doc2vec')
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_tg_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [ ]:
train_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_train, 200)
validation_vecs_ugdbow_tgdmm = get_concat_vectors(model_ug_dbow,model_tg_dmm, x_validation, 200)

In [ ]:
clf = LogisticRegression()
clf.fit(train_vecs_ugdbow_tgdmm, y_train)

In [ ]:
clf.score(validation_vecs_ugdbow_tgdmm, y_validation)

In [ ]:
#RF
clf2 = RandomForestClassifier(n_estimators=980)
clf2.fit(train_vecs_ugdbow_tgdmm, y_train)
clf2.score(validation_vecs_ugdbow_tgdmm, y_validation)

In [ ]:
#tree
# creates a ML model based on parameters
clf3 = tree.DecisionTreeClassifier(criterion='gini') 
clf3.fit(train_vecs_ugdbow_tgdmm, y_train)
clf3.score(validation_vecs_ugdbow_tgdmm, y_validation)

In [ ]:
#SVM
clf4 = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='linear'))
clf4.fit(train_vecs_ugdbow_tgdmm, y_train)
clf4.score(validation_vecs_ugdbow_tgdmm, y_validation)

In [ ]:
# clf5 = KNeighborsClassifier(n_neighbors=5)
clf5.fit(train_vecs_ugdbow_tgdmm, y_train)
clf5.score(validation_vecs_ugdbow_tgdmm, y_validation)